In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd 

from tqdm import tqdm
from utils import *

In [3]:
train_set, train_labels, test_set, test_labels = load_and_preprocess_data(path='./data/CIFAR10')

Files already downloaded and verified
Files already downloaded and verified


In [4]:
mu = 1e-3
rho = 1e-4

input_size = 32 * 32 * 3
hidden_size = 2048
output_size = 10

In [5]:
def train_network(model: NeuralNetwork, train_set, train_labels, num_epochs):
    N_samples = train_set.shape[0]

    # Add accuracy and loss tracking

    for epoch in range(num_epochs):
        
        data_idx = np.random.permutation(N_samples)
        running_epoch_loss = 0
        running_predictions = []
        running_labels = []
        
        pbar = tqdm(total=N_samples)
        for iter, idx in enumerate(data_idx):
            h = train_set[idx]
            gamma = transform_label(train_labels[idx])

            gamma_hat = model.forward(h)
            loss = cross_entropy_loss(gamma_hat, gamma, model.rho, model.weights())
            model.backward(gamma_hat, gamma)

            running_epoch_loss += loss
    
            running_predictions.append(np.argmax(gamma_hat))
            running_labels.append(train_labels[idx])

            pbar.update(1)
            pbar.set_description(f'Train loss: {loss:.4f}')
        pbar.close()

        epoch_loss = running_epoch_loss / N_samples
        epoch_accuracy = np.mean(np.array(running_predictions) == np.array(running_labels))

        print(f"Epoch: {epoch + 1} | Train Loss {epoch_loss:.4f} | Train Accuracy: {epoch_accuracy * 100:.2f}%")

    model.normalize_weights()

In [6]:
def evaluate(model: NeuralNetwork, dataset, labels):
    N_samples = dataset.shape[0]

    running_predictions = []
    running_labels = []

    pbar = tqdm(total=N_samples)
    for idx in range(N_samples):
        h = dataset[idx]
        gamma = transform_label(labels[idx])
        gamma_hat = model.forward(h)

        running_predictions.append(np.argmax(gamma_hat))
        running_labels.append(labels[idx])

        pbar.update(1)
        pbar.set_description("Evaluating...")
    pbar.close()

    num_misclassified = np.sum(np.array(running_predictions) != np.array(running_labels))
    error = num_misclassified / N_samples
    accuracy = 1 - error

    print(f"Accuracy: {accuracy * 100:.2f}% | Error: {error * 100:.2f}% | Misclassified: {num_misclassified}")

    return accuracy, error, num_misclassified


### Training without dropout

In [7]:
model = NeuralNetwork(input_size, hidden_size, output_size, ReLU, d_ReLU, mu, rho)

In [8]:
train_network(model, train_set[:5000], train_labels[:5000], num_epochs=5)

Train loss: 2.3248: 100%|██████████| 5000/5000 [05:30<00:00, 15.14it/s]


Epoch: 1 | Train Loss 2.1293 | Train Accuracy: 27.32%


Train loss: 0.9243: 100%|██████████| 5000/5000 [06:18<00:00, 13.21it/s]  


Epoch: 2 | Train Loss 1.8460 | Train Accuracy: 39.04%


Train loss: 1.1782: 100%|██████████| 5000/5000 [05:10<00:00, 16.12it/s]


Epoch: 3 | Train Loss 1.7118 | Train Accuracy: 43.66%


Train loss: 0.8550: 100%|██████████| 5000/5000 [05:09<00:00, 16.17it/s]


Epoch: 4 | Train Loss 1.6007 | Train Accuracy: 48.14%


Train loss: 2.8094: 100%|██████████| 5000/5000 [04:58<00:00, 16.73it/s]

Epoch: 5 | Train Loss 1.5025 | Train Accuracy: 51.26%


In [9]:
train_acc, train_err, train_misc = evaluate(model, train_set[:5000], train_labels[:5000])

Evaluating...: 100%|██████████| 5000/5000 [00:33<00:00, 151.11it/s]

Accuracy: 54.14% | Error: 45.86% | Misclassified: 2293


In [10]:
test_acc, test_err, test_misc = evaluate(model, test_set, test_labels)

Evaluating...: 100%|██████████| 10000/10000 [01:43<00:00, 96.63it/s]

Accuracy: 39.86% | Error: 60.14% | Misclassified: 6014


### Training with dropout

In [7]:
model = NeuralNetwork(input_size, hidden_size, output_size, ReLU, d_ReLU, mu, rho, p_dropout=[0.1, 0.5, 0.5])

In [8]:
train_network(model, train_set[:5000], train_labels[:5000], num_epochs=8)

Train loss: 2.6504: 100%|██████████| 5000/5000 [05:14<00:00, 15.92it/s]


Epoch: 1 | Train Loss 2.3851 | Train Accuracy: 16.24%


Train loss: 0.8335: 100%|██████████| 5000/5000 [05:12<00:00, 15.98it/s]


Epoch: 2 | Train Loss 2.1594 | Train Accuracy: 25.40%


Train loss: 2.5758: 100%|██████████| 5000/5000 [05:07<00:00, 16.27it/s]


...

Epoch: 6 | Train Loss 1.8718 | Train Accuracy: 36.66%


Train loss: 1.6974: 100%|██████████| 5000/5000 [05:07<00:00, 16.28it/s]


Epoch: 7 | Train Loss 1.8381 | Train Accuracy: 38.66%


Train loss: 1.7111: 100%|██████████| 5000/5000 [05:18<00:00, 15.71it/s]


Epoch: 8 | Train Loss 1.7960 | Train Accuracy: 40.80%


In [9]:
train_acc_do, train_err_do, train_misc_do = evaluate(model, train_set[:5000], train_labels[:5000])

Evaluating...: 100%|██████████| 5000/5000 [00:37<00:00, 132.26it/s]

Accuracy: 27.40% | Error: 72.60% | Misclassified: 3630


In [10]:
test_acc_do, test_err_do, test_misc_do = evaluate(model, test_set, test_labels)

Evaluating...: 100%|██████████| 10000/10000 [01:13<00:00, 136.36it/s]

Accuracy: 26.45% | Error: 73.55% | Misclassified: 7355


### Tables

In [17]:
pd.DataFrame([
    {
        "Setting": "w/o Dropout",
        "Test Error [%]": test_err,
        "# Test Misclassified": test_misc,
        "Train Error [%]": train_err,
        "# Train Misclassified": train_misc,
    },
    {
        "Setting": "w/ Dropout",
        "Test Error [%]": test_err_do,
        "# Test Misclassified": test_misc_do,
        "Train Error [%]": train_err_do,
        "# Train Misclassified": train_misc_do,
    }

])

,Setting,Test Error [%],# Test Misclassified,Train Error [%],# Train Misclassified
0,w/o Dropout,0.6014,6014,0.4586,2293
1,w/ Dropout,0.7355,7355,0.7260,3630
